# Versão 4 dos testes realizados para o modelo


In [37]:
import numpy as np
import pandas as pd 

import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 

import string
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn import metrics

from time import time

import warnings
warnings.filterwarnings("ignore")

In [38]:
# Lê as propostas com o Pandas
propostas = pd.read_csv("../../production/data_extraction/propostas.csv")

propostas  # Impressão do DataFrame de propostas

,Categoria,Texto
0,Turismo,Turismo: esse é o Destino. <p><strong>Objetivo...
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. <p><stron...
2,Agricultura e Pecuária,Agropecuária Sustentável. <p>Objetivo: Contrib...
3,Saúde,Atenção Primária à Saúde. <p>Fortalecer a Aten...
4,Saúde,Atenção Especializada à Saúde. <p>Ampliar o ac...
...,...,...
8431,Saúde,Microchipagem de animais de companhia. Microch...
8432,Secretaria Geral da Presidência da República,"Direito a atendimento eficiente, é um direito..."
8433,Previdência Social,Rever Reforma da Previdência. Rever a reforma ...
8434,Meio Ambiente e Mudança do Clima,Preservação da Serra do Espinhaço. A Serra do ...


In [39]:
# Expressões regulares para remoção de poluição de dados nos textos 
# REGX_URL = r"https?://[A-Za-z0-9./\-]+" # Regex for URLs
REGX_HTML = r"<[^<]+?>" # Regex for HTML tags
REGX_ENDING = r'Órgão Responsável:.+' # Regex for the part of the text

In [40]:
# Função para pré-processamento
def preprocessing(text):
  # text = text.lower()

  text = re.sub(REGX_HTML, '', text)  # Removendo tags HTML
  # text = re.sub(REGX_URL, '', text) # Revomendo URLs
  text = re.sub(REGX_ENDING, '', text)

  # tokens = [t.lemma_ for t in nlp(text) if t not in STOP_WORDS and not t.is_punct]

  return text

#  Ajustando textos da coluna Corpo
propostas['Texto'] = propostas['Texto'].apply(preprocessing)
propostas.head()

,Categoria,Texto
0,Turismo,Turismo: esse é o Destino. Objetivo: Posiciona...
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. Objetivo:...
2,Agricultura e Pecuária,Agropecuária Sustentável. Objetivo: Contribuir...
3,Saúde,Atenção Primária à Saúde. Fortalecer a Atenção...
4,Saúde,Atenção Especializada à Saúde. Ampliar o acess...


In [41]:
print(propostas.Categoria.value_counts())

Categoria
Saúde                                                             1368
Educação                                                          1261
Direitos Humanos e Cidadania                                       528
Meio Ambiente e Mudança do Clima                                   497
Transportes                                                        473
Desenvolvimento e Assistência Social, Família e Combate à Fome     418
Trabalho e Emprego                                                 377
Justiça e Segurança Pública                                        342
Cidades                                                            294
Gestão e Inovação em Serviços Públicos                             284
Desenvolvimento Agrário e Agricultura Familiar                     255
Previdência Social                                                 249
Cultura                                                            224
Mulheres                                                           

# Realizando a criacao de novas propostas e verificando a performance no modelo



In [42]:
# Lê as propostas com o Pandas

simuladas = pd.read_csv("propostas_simuladas.csv")
simuladas  # Impressão do DataFrame de propostas

,Categoria,Proposta
0,Casa Civil,É ótimo ver a conscientização sobre igualdade ...
1,Casa Civil,Tornar a Casa Civil mais ecológica e sustentáv...
2,Casa Civil,Criar um programa de parcerias com startups na...
3,Casa Civil,Vamos estabelecer metas ambiciosas para reduzi...
4,Casa Civil,Vamos criar um comitê de ética interno na Casa...
...,...,...
366,Comunicações,O serviço postal desempenha um papel essencial...
367,Comunicações,A imprensa local desempenha um papel vital na ...
368,Comunicações,Fusões e aquisições no setor de mídia podem te...
369,Comunicações,"Para capacitar os cidadãos na era digital, é c..."


In [43]:
propostas = pd.DataFrame(propostas)
propostas_simuladas = pd.DataFrame(simuladas)
novas_propostas = pd.concat([propostas,propostas_simuladas],ignore_index=True)

In [44]:
print(novas_propostas.Categoria.value_counts())

Categoria
Saúde                                                             1368
Educação                                                          1261
Direitos Humanos e Cidadania                                       528
Meio Ambiente e Mudança do Clima                                   497
Transportes                                                        473
Desenvolvimento e Assistência Social, Família e Combate à Fome     418
Trabalho e Emprego                                                 377
Justiça e Segurança Pública                                        342
Cidades                                                            294
Gestão e Inovação em Serviços Públicos                             284
Desenvolvimento Agrário e Agricultura Familiar                     255
Previdência Social                                                 249
Cultura                                                            224
Mulheres                                                           

# Pre Processamento


In [45]:
#Criando índices para as categorias
cats = novas_propostas['Categoria'].unique() # Pegando cada categoria única
cats = dict(enumerate(cats, 0)) # Convertendo para dict (com índices enumerados)
cats = {v:k for k,v in cats.items()}  # Trocando chaves e valores

novas_propostas['id_cats'] = novas_propostas['Categoria'].map(cats) # Inserindo coluna de índices das cats
novas_propostas.head()

,Categoria,Texto,Proposta,id_cats
0,Turismo,Turismo: esse é o Destino. Objetivo: Posiciona...,NaN,0
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. Objetivo:...,NaN,1
2,Agricultura e Pecuária,Agropecuária Sustentável. Objetivo: Contribuir...,NaN,2
3,Saúde,Atenção Primária à Saúde. Fortalecer a Atenção...,NaN,3
4,Saúde,Atenção Especializada à Saúde. Ampliar o acess...,NaN,3


In [46]:
novas_propostas['Texto'][0]

'Turismo: esse é o Destino. Objetivo: Posicionar o turismo como vetor de desenvolvimento sustentável e aumentar a competitividade dos destinos e produtos turísticos brasileiros, democratizando o acesso à atividade turística aos cidadãos brasileiros.'

In [47]:
import nltk
import spacy
nlp = spacy.load('pt_core_news_sm')  

nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('portuguese')

lemmatizer = WordNetLemmatizer()

def clean(doc):
    
    if isinstance(doc, float) and np.isnan(doc):
        return ''
    
    text_no_namedentities = []
    document = nlp(doc)
    ents = [e.text for e in document.ents]
    for item in document:
        if item.text in ents:
            pass
        else:
            text_no_namedentities.append(item.text)
    doc = (" ".join(text_no_namedentities))

    doc = doc.lower().strip()
    doc = doc.replace("</br>", " ") 
    doc = doc.replace("-", " ") 
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])    
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])
    return doc


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leandro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
clean(novas_propostas['Texto'][0])

'turismo objetivo posicionar turismo vetor desenvolvimento sustentável aumentar competitividade destinos produtos turísticos brasileiros democratizando acesso atividade turística cidadãos brasileiros'

In [49]:
novas_propostas['Texto'] = novas_propostas['Texto'].apply(clean)
novas_propostas.head()

,Categoria,Texto,Proposta,id_cats
0,Turismo,turismo objetivo posicionar turismo vetor dese...,NaN,0
1,Desenvolvimento Agrário e Agricultura Familiar,agricultura familiar objetivo fortalecer agric...,NaN,1
2,Agricultura e Pecuária,agropecuária objetivo contribuir desenvolvimen...,NaN,2
3,Saúde,atenção primária fortalecer atenção primária a...,NaN,3
4,Saúde,atenção especializada ampliar acesso ações ser...,NaN,3


## TF-IDF Vectorizer

![TF-IDF](https://miro.medium.com/max/3136/1*ruCawEw0--m2SeHmAQooJQ.jpeg)

In [50]:
docs = list(novas_propostas['Texto'])
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 20000) 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)
docs = tfidf_vectorizer_vectors.toarray()

In [51]:
X = docs 
y = novas_propostas['id_cats']
print(X.shape, y.shape)


(8807, 20000) (8807,)


In [52]:
fig = go.Figure([go.Bar(x=y.value_counts().index, y=y.value_counts().tolist())])
fig.update_layout(
    title="Quantidade de propostas",
    xaxis_title="Propostas",
    yaxis_title="quantidade")
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

# Train test Split

In [53]:
SEED=123
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7045, 20000) (7045,)
(1762, 20000) (1762,)


In [54]:
target_names = [
    'Saúde', 'Educação', 'Direitos Humanos e Cidadania', 'Meio Ambiente e Mudança do Clima',
    'Transportes', 'Desenvolvimento e Assistência Social, Família e Combate à Fome',
    'Trabalho e Emprego', 'Justiça e Segurança Pública', 'Cidades',
    'Gestão e Inovação em Serviços Públicos', 'Desenvolvimento Agrário e Agricultura Familiar',
    'Previdência Social', 'Cultura', 'Mulheres', 'Ciência, Tecnologia e Inovação',
    'Fazenda', 'Planejamento e Orçamento', 'Turismo',
    'Desenvolvimento, Indústria, Comércio e Serviços', 'Integração e Desenvolvimento Regional',
    'Agricultura e Pecuária', 'Minas e Energia', 'Igualdade Racial',
    'Secretaria Geral da Presidência da República', 'Esporte', 'Casa Civil',
    'Defesa', 'Comunicações', 'Comunicação Social', 'Relações Institucionais',
    'Povos Indígenas', 'Pesca e Aquicultura', 'Controladoria-Geral da União',
    'Outros', 'Segurança Institucional', 'Banco Central', 'Portos e Aeroportos',
    'Advocacia-Geral da União', 'Relações Exteriores'
]

## Balanceando dados

## Classificação com Naive Bayes 

### Gaussian Naive Bayes


In [55]:
gnb = GaussianNB() 
%time gnb.fit(X_train, y_train)

y_pred_train = gnb.predict(X_train)
y_pred_test = gnb.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 1.27 s, sys: 692 ms, total: 1.96 s
Wall time: 1.96 s

Training Accuracy score: 0.9104329311568489
Testing Accuracy score: 0.32009080590238365


In [57]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.40      0.08      0.14        24
                                                      Educação       0.50      0.25      0.34        51
                                  Direitos Humanos e Cidadania       0.50      0.33      0.40        21
                              Meio Ambiente e Mudança do Clima       0.37      0.58      0.45       274
                                                   Transportes       0.00      0.00      0.00        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.25      0.19      0.22        84
                                            Trabalho e Emprego       0.43      0.15      0.22        20
                                   Justiça e Segurança Pública       0.17      0.11      0.13        75
                                                       Cidades 

### Multinomial Naive Bayes

In [58]:
mnb = MultinomialNB() 
%time mnb.fit(X_train, y_train)

y_pred_train = mnb.predict(X_train)
y_pred_test = mnb.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 1.21 s, sys: 192 ms, total: 1.4 s
Wall time: 323 ms

Training Accuracy score: 0.3797019162526615
Testing Accuracy score: 0.3246311010215664


In [59]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.00      0.00      0.00        24
                                                      Educação       0.60      0.06      0.11        51
                                  Direitos Humanos e Cidadania       0.00      0.00      0.00        21
                              Meio Ambiente e Mudança do Clima       0.30      0.91      0.45       274
                                                   Transportes       0.00      0.00      0.00        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.00      0.00      0.00        84
                                            Trabalho e Emprego       0.00      0.00      0.00        20
                                   Justiça e Segurança Pública       0.00      0.00      0.00        75
                                                       Cidades 

## Logistic Regression Classifier

In [60]:
lr = LogisticRegression(random_state=SEED)
%time lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 6min 17s, sys: 3min 35s, total: 9min 52s
Wall time: 1min 20s

Training Accuracy score: 0.6894251242015614
Testing Accuracy score: 0.5175936435868331


In [61]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.92      0.46      0.61        24
                                                      Educação       0.62      0.67      0.64        51
                                  Direitos Humanos e Cidadania       1.00      0.10      0.17        21
                              Meio Ambiente e Mudança do Clima       0.41      0.86      0.55       274
                                                   Transportes       0.00      0.00      0.00        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.35      0.23      0.28        84
                                            Trabalho e Emprego       0.88      0.35      0.50        20
                                   Justiça e Segurança Pública       0.51      0.44      0.47        75
                                                       Cidades 

## Support Vector Machines

In [62]:
svc =  LinearSVC(class_weight='balanced') 
%time svc.fit(X_train, y_train)

y_pred_train = svc.predict(X_train)
y_pred_test = svc.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 1.2 s, sys: 532 ms, total: 1.73 s
Wall time: 1.12 s

Training Accuracy score: 0.9388218594748048
Testing Accuracy score: 0.5317820658342792


In [63]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.73      0.67      0.70        24
                                                      Educação       0.56      0.65      0.60        51
                                  Direitos Humanos e Cidadania       0.42      0.38      0.40        21
                              Meio Ambiente e Mudança do Clima       0.76      0.68      0.71       274
                                                   Transportes       0.50      0.38      0.43        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.41      0.29      0.34        84
                                            Trabalho e Emprego       0.74      0.70      0.72        20
                                   Justiça e Segurança Pública       0.43      0.52      0.47        75
                                                       Cidades 

## Decision Tree Classifier

In [64]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=SEED)
%time dt.fit(X_train, y_train)

y_pred_train = dt.predict(X_train)
y_pred_test = dt.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

CPU times: user 41 s, sys: 365 ms, total: 41.4 s
Wall time: 41.1 s

Training Accuracy score: 0.958694109297374
Testing Accuracy score: 0.39330306469920545


In [65]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.60      0.50      0.55        24
                                                      Educação       0.28      0.29      0.29        51
                                  Direitos Humanos e Cidadania       0.10      0.10      0.10        21
                              Meio Ambiente e Mudança do Clima       0.55      0.61      0.58       274
                                                   Transportes       0.00      0.00      0.00        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.29      0.27      0.28        84
                                            Trabalho e Emprego       0.55      0.55      0.55        20
                                   Justiça e Segurança Pública       0.24      0.23      0.23        75
                                                       Cidades 

## Ensembling

In [66]:
from sklearn.ensemble import VotingClassifier

classifiers = [('Decision Tree', dt),
               ('Logistic Regression', lr),
                ('Naive Bayes', gnb)
              ]
vc = VotingClassifier(estimators=classifiers)
# Fit 'vc' to the traing set and predict test set labels
vc.fit(X_train, y_train)
y_pred_train=vc.predict(X_train)
y_pred_test = vc.predict(X_test)
print("Training Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))

Training Accuracy score: 0.9508871540099362
Testing Accuracy score: 0.47219069239500566


In [67]:
print(classification_report(y_test, y_pred_test,target_names=target_names))

                                                                precision    recall  f1-score   support

                                                         Saúde       0.68      0.54      0.60        24
                                                      Educação       0.43      0.65      0.52        51
                                  Direitos Humanos e Cidadania       0.27      0.19      0.22        21
                              Meio Ambiente e Mudança do Clima       0.40      0.87      0.55       274
                                                   Transportes       0.00      0.00      0.00        13
Desenvolvimento e Assistência Social, Família e Combate à Fome       0.27      0.25      0.26        84
                                            Trabalho e Emprego       0.69      0.45      0.55        20
                                   Justiça e Segurança Pública       0.36      0.28      0.31        75
                                                       Cidades 

In [68]:
predictions = pd.Series(vc.predict(X), name="Categoria")
results = pd.concat([predictions],axis=1)
results.to_csv("novas_propostas_resultado.csv",index=False)